In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame()
attributes= pd.DataFrame()

base_site = "https://kolesa.kz/cars/?"
inner_site = "https://kolesa.kz/a/show/"

for i in range(1000):
    response = requests.get(base_site + f"page={i}")
    print(response.status_code)

    if response.status_code == 200:
        html = response.content
        soup = BeautifulSoup(html, 'lxml')
    
        divs = soup.find_all('div', {'class': 'a-info-side col-right-list'})
        a = [divs.find('a', class_='list-link ddl_product_link') for divs in divs]
        ID = [a['data-product-id'] for a in a]
        car_name = [divs.find('a').string for divs in divs]
        price = [divs.find('span', class_="price").text for divs in divs]
        price = [int(''.join(price.split('                    ')[1].split('\xa0')[:-1])) for price in price]
        description = str(item.find('p', class_='a-card__description').get_text(strip=True))
    
        for k in ID:
            # print(k)
            response2 = requests.get(inner_site+f"{k}")
            html2 = response2.content
            soup2 = BeautifulSoup(html2, 'lxml')
            h1 = soup2.find('h1', {'class': 'offer__title'})
        
            brand = h1.find('span', itemprop="brand").string
            if h1.find('span', itemprop="name") != None:
                make = h1.find('span', itemprop="name").string
            year = int(h1.find('span', class_="year").string)

            div = soup2.find('div', {'class': 'offer__parameters'})   
            d={}
            for i in div.find_all('dt'):
                if i.find('span') != None:
                    d[f"{i.find('span').string}"]=div.find('dt', title=i['title']).parent.find('dd').string.strip()
            d.update({'brand': brand, 'model': make, 'year': year})
            data1 = [d]
            # print(data1)
            attributes = attributes.append(data1, ignore_index=True, sort=False)
    
        new_data=pd.DataFrame()
        new_data['description'] = description
        new_data['price'] = price
        new_data['url'] = ID
        data = pd.concat([data, new_data], axis=0, ignore_index=True)
    
    data = data.join(attributes)
    data.to_excel(r'.\data\cars.csv', sep=';', index = False)

In [9]:
data.head()

,brand,description,model,price,url,year,Город,Коробка передач,Кузов,"Объем двигателя, л",Привод,Растаможен,Руль,Цвет,Пробег,Наличие,VIN
0,Mercedes-Benz,"полный электропакет , налог уплачен, техосмот...",GL 550,5800000,https://kolesa.kz/a/show/10235750,2008,Актау,типтроник,внедорожник,5.5 (бензин),полный привод,Да,слева,черный металлик,NaN,NaN,NaN
1,BMW,"тонировка , обогрев зеркал, хрустальная оптик...",X5,5500000,https://kolesa.kz/a/show/10269314,2009,Нур-Султан (Астана),автомат,кроссовер,4.8 (бензин),полный привод,Да,слева,черный,170 км,NaN,NaN
2,Mercedes-Benz,"лонг, состояние нового авто немец газ-бензин п...",S 350,7770000,https://kolesa.kz/a/show/1063798,2006,Алматы,типтроник,седан,3.5 (бензин),NaN,Да,слева,черный,60 000 км,NaN,NaN
3,Mitsubishi,"литые диски, тонировка, люк , хрустальная опт...",Outlander,3200000,https://kolesa.kz/a/show/10695701,2003,Семей,типтроник,кроссовер,2.4 (бензин),полный привод,Да,слева,серебристый металлик,120 000 км,NaN,NaN
4,Ретро-автомобили,"налог уплачен, техосмотр пройден, вложений не ...","ГАЗ 21 ""Волга""",3900000,https://kolesa.kz/a/show/10852142,1970,Алматы,механика,седан,2.4 (бензин),задний привод,Да,слева,черный,1 200 км,NaN,NaN
